In [1]:
import os, sys
# enable absolute paths transversal (from notebooks folder to src folder)
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from datetime import datetime, timedelta
import streamlit as st
from pypfopt import expected_returns, EfficientFrontier
import pandas as pd
import numpy as np

import src.utils as utils
import src.macro.calculate as calculate
import src.macro.plot as plot

tickers = ['AAPL','AMZN','NVDA','MMC','GOOG','MSFT','BTC-USD','ETH-USD','XOM','BAC','V','GOLD','^GSPC']
start_date = '2014-01-01'
end_date = datetime.now() - timedelta(1)

# just grabbing the 'Adj Close' column
stock_data = utils.get_stock_data(tickers, start_date, end_date)

sp500_data = stock_data['^GSPC']
stock_data = stock_data.drop(['^GSPC'], axis=1)

stock_data.tail()

"""similar to portfolio/display.py, set the portfolio weights and calculate the performance
"""
risk_free_rate = 0.04
mu = expected_returns.mean_historical_return(stock_data)
S = utils.calculate_covariance_matrix(stock_data)

# Calculating the cumulative monthly returns of a portfolio of stocks with given weights:
min_risk, max_risk = utils.calculate_risk_extents(mu, S, risk_free_rate)
risk = (max_risk + min_risk) / 2

ef = EfficientFrontier(mu, S)
ef.efficient_risk(risk)
weights = ef.clean_weights()
weights = pd.Series(weights).reindex(stock_data.columns)
ef_returns, ef_volatility, ef_sharpe = ef.portfolio_performance(risk_free_rate)
print(f'ef weights:\n{weights}')
print(f'ef performance: {ef_returns, ef_volatility, ef_sharpe}')



2023-06-03 05:29:14.805 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-03 05:29:14.807 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-03 05:29:14.808 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-03 05:29:14.809 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-03 05:29:14.810 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-03 05:29:14.810 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-03 05:29:14.811 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-03 05:29:14.812 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-

[*********************100%***********************]  13 of 13 completed
Expected annual return: 28.0%
Annual volatility: 23.3%
Sharpe Ratio: 1.11
ef weights:
AAPL       0.11997
AMZN       0.00000
BAC        0.00000
BTC-USD    0.15161
ETH-USD    0.00000
GOLD       0.00000
GOOG       0.00000
MMC        0.27981
MSFT       0.09622
NVDA       0.35239
V          0.00000
XOM        0.00000
dtype: float64
ef performance: (0.27988419701883627, 0.23321841467899954, 1.1143382368692427)


In [2]:
%pip install python-dotenv




Note: you may need to restart the kernel to use updated packages.


In [19]:
from dotenv import load_dotenv
load_dotenv()

print(os.environ.get('FRED_API_KEY'))
print(os.environ.get('FMP_API_KEY'))
print(os.environ.get('NASDAQ_API_KEY'))

6909a5f9244a34fc5b3724c919a2ddd0
e12908f5601c2198927fecc7d3dcb1ea
i3rjXmcrBdZ1sLCv8Pij


In [18]:
%pip install pip install Nasdaq-Data-Link
%pip install quandl

Note: you may need to restart the kernel to use updated packages.
  Using cached Quandl-3.7.0-py2.py3-none-any.whl (26 kB)
Note: you may need to restart the kernel to use updated packages.


In [20]:
import quandl

us_m2_data = quandl.get("FED/M2_N_M", authtoken=os.environ.get('NASDAQ_API_KEY'))
print(f'head: {us_m2_data.head()}')
print(f'tail: {us_m2_data.tail()}')
print(f'description: {us_m2_data.describe()}')

head:             Value
Date             
1959-01-31  289.8
1959-02-28  287.7
1959-03-31  287.9
1959-04-30  290.2
1959-05-31  290.2
tail:               Value
Date               
2022-12-31  21433.1
2023-01-31  21280.9
2023-02-28  21113.2
2023-03-31  21006.2
2023-04-30  20827.1
description:               Value
count    772.000000
mean    5017.158549
std     5260.669388
min      287.700000
25%      908.525000
50%     3312.450000
75%     7181.600000
max    21855.800000


In [16]:


from fredapi import Fred

import requests

import logging
logging.basicConfig(level=logging.WARNING, format='%(asctime)s (%(levelname)s):  %(module)s.%(funcName)s - %(message)s')

# Set up logger for a specific module to a different level
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# get the macro factors, eg, interest rate, inflation, money supply, etc that we want to analyze
user_macro_input = calculate.get_macro_factor_defaults()
print(f'user_macro_input: {user_macro_input}')
            
def get_historical_macro_data(start_date, end_date):
    fred = Fred(api_key=os.environ.get('FRED_API_KEY'))
    print(f"Fetching macroeconomic data from FRED from {start_date} to {end_date}")
    # Get macroeconomic data
    us_interest_rate = fred.get_series('GS10', start_date, end_date)  # 10-Year Treasury Constant Maturity Rate
    us_inflation = fred.get_series('T10YIE', start_date, end_date)  # 10-Year Breakeven Inflation Rate
    us_m2_money_supply = fred.get_series('M2', start_date, end_date)  # M2 Money Stock
    china_m2_money_supply = fred.get_series('MYAGM2CNM189N', start_date, end_date)  # China M2 Money Supply
    
    """ need to get a paid subscription to get the ecomomic data from FMP
    fmp_api_key = os.environ.get('FMP_API_KEY')
    print(f"Fetching economic data from FMP from {start_date} to {end_date} with api key {fmp_api_key}")
    base_url = "https://financialmodelingprep.com/api/v4/economic"

    indicators = ["GDP", "realGDP", "federalFunds", "CPI", "inflationRate", "inflation", "retailSales", "consumerSentiment", "unemploymentRate", "initialClaims"]
    
    for indicator in indicators:
        params = {"apikey": fmp_api_key, "from": start_date, "to": end_date, "name": indicator}
        response = requests.get(f"{base_url}", params=params)
        print(f"response for {indicator} was:\n{response}")
        data = response.json()
        df = pd.DataFrame(data)
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        df = df[start_date:end_date]
        macroeconomic_data = pd.concat([macroeconomic_data, df], axis=1)
    """

    logger.debug("macroeconomic_data.head():\n{}".format(macroeconomic_data.head()))
    macroeconomic_data.columns = calculate.get_macro_factor_list()
    
    return macroeconomic_data

macroeconomic_data = get_historical_macro_data(start_date, end_date)

figs_to_plot = plot.plot_historical_macro_data(macroeconomic_data)

for fig in figs_to_plot:
    fig.show()

user_macro_input: {'US Interest Rate': 0.01, 'US Inflation Rate': 0.02, 'US M2 Money Supply': 0.1, 'China M2 Money Supply': 0.15}
Fetching macroeconomic data from FRED from 2014-01-01 to 2023-06-02 05:29:14.916319
Fetching economic data from FMP from 2014-01-01 to 2023-06-02 05:29:14.916319 with api key e12908f5601c2198927fecc7d3dcb1ea
response for GDP was:
<Response [403]>


ValueError: If using all scalar values, you must pass an index

In [5]:
import src.macro.calculate as calculate
import src.macro.plot as plot

# bring the macro data into the same format as the portfolio data as a new df (monthly basis), clean it and do some summary calcs
combined_data = calculate.clean_and_combine_macro_data(stock_data, weights, macroeconomic_data)
print(f'combined_data.head():\n{combined_data.head()}')

fig1, fig2 = plot.plot_historical_portfolio_performance(combined_data)
fig1.show()
fig2.show()

daily_returns tail:
                AAPL      AMZN       BAC   BTC-USD   ETH-USD      GOLD   
Date                                                                     
2023-05-28  0.000000  0.000000  0.000000  0.045306  0.043578  0.000000  \
2023-05-29  0.000000  0.000000  0.000000 -0.012097 -0.009334  0.000000   
2023-05-30  0.010660  0.012905 -0.001766 -0.001569  0.004199 -0.006513   
2023-05-31 -0.000282 -0.008877 -0.016631 -0.017424 -0.014148  0.005959   
2023-06-01  0.016023  0.018162  0.007617 -0.014684 -0.006365  0.033768   

                GOOG       MMC      MSFT      NVDA         V       XOM  
Date                                                                    
2023-05-28  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
2023-05-29  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
2023-05-30 -0.006298  0.006292 -0.005047  0.029913 -0.014977 -0.008860  
2023-05-31 -0.010189 -0.006540 -0.008514 -0.056767 -0.002752 -0.017878  
2023-06-01  0.008106  0

In [6]:
# plot the combined macro and portfolio data
macro_vs_portfolio_returns_plots = plot.plot_macro_vs_portfolio_performance(combined_data)

for macro_vs_portfolio_returns_plot in macro_vs_portfolio_returns_plots:
    macro_vs_portfolio_returns_plot.show()

In [7]:
# calculate the correlation between the macro factors and the portfolio returns (monthly basis)
# model data will have less data points than combined_data because we need to drop any rows with NaNs for the linear regression
# If the correlation is positive, it means that when the interest rate or inflation increases, our portfolio returns also tend to increase.
#
# Past performance is not indicative of future results, and this analysis assumes that the relationships between these variables and portfolio 
# returns will remain constant in the future, which may not be the case.
#
# The model also assumes a linear relationship between the predictors and the response variable. There could be a non-linear relationship 
# between them which cannot be captured by this model

# hypothesis that these factors may impact **cumulative** performance
cumulative_macro_factors = ['cumulative_inflation', 'US M2 Money Supply', 'China M2 Money Supply']
y_cumulative = 'weighted_portfolio_cumulative_returns'
cumulative_models, cumulative_model_data = calculate.calculate_linear_regression_models_from_macro_data_per_factor(combined_data, cumulative_macro_factors, y_cumulative)
cumulative_performance_predictions = calculate.predict_portfolio_returns_from_user_macro_input(user_macro_input, cumulative_models)

# hypothesis that these factors may impact **month to month** performance
rate_macro_factors = ['US Interest Rate', 'US Inflation Rate', 'US M2 Money Supply', 'China M2 Money Supply']
y_rate = 'weighted_portfolio_returns_monthly'
rate_models, rate_model_data = calculate.calculate_linear_regression_models_from_macro_data_per_factor(combined_data, rate_macro_factors, y_rate)
rate_performance_predictions = calculate.predict_portfolio_returns_from_user_macro_input(user_macro_input, rate_models)

In [8]:
def display_regression_formula(model, factor_name, y):
    # Get the intercept and coefficient
    intercept = model.intercept_
    coef = model.coef_[0]
    
    # Format the formula string
    formula = f"{y} = {intercept:.4f} + ({coef:.4f} * {factor_name})"
    
    # Display the formula
    print(formula)
    
# plot the linear regression model
for factor, model in cumulative_models.items():
    print(display_regression_formula(model, factor, "Cumulative Returns"))
    prediction = None
    if factor in cumulative_performance_predictions:
        prediction = {factor: user_macro_input[factor], 'prediction': cumulative_performance_predictions[factor][0]}
    fig = plot.plot_linear_regression_v_single_model(model, cumulative_model_data, factor, "weighted_portfolio_cumulative_returns", prediction=prediction)
    fig.show()

    
for factor, model in rate_models.items():
    # ... (plot the regression here)
    print(display_regression_formula(model, factor, "Monthly Returns"))
    prediction = None
    if factor in rate_performance_predictions:
        prediction = {factor: user_macro_input[factor], 'prediction': rate_performance_predictions[factor][0]}
    fig = plot.plot_linear_regression_v_single_model(model, rate_model_data, factor, "weighted_portfolio_returns_monthly", prediction=prediction)
    fig.show()

Cumulative Returns = -6.9022 + (0.1618 * cumulative_inflation)
None


Cumulative Returns = -26.4141 + (0.0022 * US M2 Money Supply)
None


Cumulative Returns = -12.9062 + (0.0000 * China M2 Money Supply)
None


Cumulative Returns = 1.7199 + (0.0736 * all_factors)
None


Monthly Returns = 0.0751 + (-0.0179 * US Interest Rate)
None


Monthly Returns = 0.0828 + (-0.0242 * US Inflation Rate)
None


Monthly Returns = -0.0327 + (0.0000 * US M2 Money Supply)
None


Monthly Returns = 0.0208 + (0.0000 * China M2 Money Supply)
None


Monthly Returns = -0.8529 + (-0.0857 * all_factors)
None
